In [1]:
!pip install transformers==4.40.1 bitsandbytes==0.43.1 accelerate==0.29.3 datasets==2.19.0 tiktoken==0.6.0 huggingface_hub==0.22.2 autotrain-advanced==0.7.77 -qqq
!pip install --upgrade huggingface-hub -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 45.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 80.4 MB/s eta 0:00

## 예제 6.2. SQL 프롬프트

In [2]:
def make_prompt(ddl, question, query=''):
    prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
{ddl}

### Question:
{question}

### SQL:
{query}"""
    return prompt
'''
먼저 명령을작성하고 필요한 데이터(함수의 입력)을 넣어준후 정답에 해당하는 SQL을 마지막에 넣는다.
학습 데이터는 정답이 채워진 형태로 사용하고 SQL을 생성할 때는 빈 문자열을 넣어준다.
데이터 예제는 책 p.213에 있으므로 이걸보고 이해해도 좋다.
'''

'\n먼저 명령을작성하고 필요한 데이터(함수의 입력)을 넣어준후 정답에 해당하는 SQL을 마지막에 넣는다.  \n학습 데이터는 정답이 채워진 형태로 사용하고 SQL을 생성할 때는 빈 문자열을 넣어준다.\n데이터 예제는 책 p.213에 있으므로 이걸보고 이해해도 좋다.\n'

## 예제 6.4. 평가를 위한 요청 jsonl 작성 함수

In [3]:
import json
import pandas as pd
from pathlib import Path

def make_requests_for_gpt_evaluation(df, filename, dir='requests'):
  if not Path(dir).exists():
      Path(dir).mkdir(parents=True) #dir 경로가 존재 안하면 폴더 만든다.
  prompts = []
  for idx, row in df.iterrows(): #입력으로 들어온 df를 전부 탐색한다.
      prompts.append("""Based on below DDL and Question, evaluate gen_sql can resolve Question. If gen_sql and gt_sql do equal job, return "yes" else return "no". Output JSON Format: {"resolve_yn": ""}""" + f"""
DDL: {row['context']}
Question: {row['question']}
gt_sql: {row['answer']}
gen_sql: {row['gen_sql']}"""
)
  jobs = [{"model": "gpt-4-turbo-preview", "response_format" : { "type": "json_object" }, "messages": [{"role": "system", "content": prompt}]} for prompt in prompts]
  with open(Path(dir, filename), "w") as f:
      for job in jobs:
          json_string = json.dumps(job)
          f.write(json_string + "\n")

## 예제 6.5. 비동기 요청 명령

In [7]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('op')

'''
이걸 실행하면 에러가 나오는데 이걸 그대로 실행하라는 의미가 아니라
OpinAI에 API 요청을 하려면 이런 데이터 형식으로 실행하라는 것을 보여주는 거임
보여주기용이라 당연히 문법이 틀림 그냥 Argument 설정해주는 방법
python api_request_parallel_processor.py \
  --requests_filepath {요청 파일 경로} \
  --save_filepath {생성할 결과 파일 경로} \
  --request_url https://api.openai.com/v1/chat/completions \
  --max_requests_per_minute 300 \
  --max_tokens_per_minute 100000 \
  --token_encoding_name cl100k_base \
  --max_attempts 5 \
  --logging_level 20'''

'\n이걸 실행하면 에러가 나오는데 이걸 그대로 실행하라는 의미가 아니라 \nOpinAI에 API 요청을 하려면 이런 데이터 형식으로 실행하라는 것을 보여주는 거임 \n보여주기용이라 당연히 문법이 틀림 그냥 Argument 설정해주는 방법\npython api_request_parallel_processor.py   --requests_filepath {요청 파일 경로}   --save_filepath {생성할 결과 파일 경로}   --request_url https://api.openai.com/v1/chat/completions   --max_requests_per_minute 300   --max_tokens_per_minute 100000   --token_encoding_name cl100k_base   --max_attempts 5   --logging_level 20'

## 예제 6.6. 결과 jsonl 파일을 csv로 변환하는 함수

In [4]:
def change_jsonl_to_csv(input_file, output_file, prompt_column="prompt", response_column="response"):
    prompts = []
    responses = []
    with open(input_file, 'r') as json_file:
        for data in json_file:
            prompts.append(json.loads(data)[0]['messages'][0]['content'])
            responses.append(json.loads(data)[1]['choices'][0]['message']['content'])

    df = pd.DataFrame({prompt_column: prompts, response_column: responses})
    df.to_csv(output_file, index=False)
    return df

## 예제 6.7. 기초 모델로 생성하기

In [5]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

def make_inference_pipeline(model_id):
  '''
  입력으로 들어온 ID에 맞는 모델과 토크나이저를 불러와 파이프라인을 생성해서 반환한다.
  '''
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
  pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
  return pipe

model_id = 'beomi/Yi-Ko-6B'
hf_pipe = make_inference_pipeline(model_id) #파이프 라인 생성

example = """당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
CREATE TABLE players (
  player_id INT PRIMARY KEY AUTO_INCREMENT,
  username VARCHAR(255) UNIQUE NOT NULL,
  email VARCHAR(255) UNIQUE NOT NULL,
  password_hash VARCHAR(255) NOT NULL,
  date_joined DATETIME NOT NULL,
  last_login DATETIME
);

### Question:
사용자 이름에 'admin'이 포함되어 있는 계정의 수를 알려주세요.

### SQL:
"""

hf_pipe(example, do_sample=False,
    return_full_text=False, max_length=512, truncation=True)
#  SELECT COUNT(*) FROM players WHERE username LIKE '%admin%';

# ### SQL 봇:
# SELECT COUNT(*) FROM players WHERE username LIKE '%admin%';

# ### SQL 봇의 결과:
# SELECT COUNT(*) FROM players WHERE username LIKE '%admin%'; (생략)

'''
위에 주석된 부분이 이 구문을 실행해서 얻는 결과이다.
하지만 우리는 맨위에 한줄만 결과로 얻으면 충분한데
뒤에 결과가 추가로 계속 생성되는 모습을 확인 할 수 있다.
그러므로 미세 조정해서 학습을 시켜야 한다.
'''

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/9.51k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00004-of-00005.safetensors:   0%|          | 0.00/2.86G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/643M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

[{'generated_text': "SELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE 

In [12]:
hf_pipe(example, do_sample=False,
    return_full_text=False, max_length=512, truncation=True)

[{'generated_text': "SELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE 

## 예제 6.8. 기초 모델 성능 측정

In [6]:
!mkdir results

In [9]:
from datasets import load_dataset
# 데이터셋 불러오기
df = load_dataset("shangrilar/ko_text2sql", "origin")['test'] #기초 모델의 성능을 평가하기 위한거라 test로 평가만 진행
df = df.to_pandas() #df를 판다스 형태로 변경
for idx, row in df.iterrows():
  prompt = make_prompt(row['context'], row['question']) #make_promplt에 넣어서 프롬프트 만들어줌
  df.loc[idx, 'prompt'] = prompt #df에 프롬프트 집어넣음
# sql 생성


In [10]:
df.head(5)

,db_id,context,question,answer,prompt
0,1,CREATE TABLE quests (\n quest_id INT PRIMARY ...,각 보상 아이템별로 보상 경험치의 합을 구해줘,"SELECT reward_items, SUM(reward_experience) AS...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
1,1,CREATE TABLE players (\n player_id INT PRIMAR...,사용자 이름에 'admin'이 포함되어 있는 계정의 수를 알려주세요.,SELECT COUNT(*) FROM players WHERE username LI...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
2,1,CREATE TABLE quests (\n quest_id INT PRIMARY ...,퀘스트 진행 상황이 100%인 퀘스트의 이름과 보상 경험치는 얼마인가요?,"SELECT q.name, q.reward_experience FROM quests...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
3,1,CREATE TABLE characters (\n character_id INT ...,경험이 5000000 이상이거나 직업이 전사인 캐릭터들의 이름은 무엇인가,SELECT name FROM characters WHERE experience >...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
4,1,CREATE TABLE characters (\n character_id INT ...,레벨이 20 이상인 플레이어의 캐릭터 이름과 해당 캐릭터의 스킬 이름을 알아보세요.,"SELECT C.name, ST.skill_name FROM characters A...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...


In [ ]:
gen_sqls = hf_pipe(df['prompt'].tolist(), do_sample=False,  #df에 있는 프롬프트 부분만 입력으로 넣어서 답변을 생성함
                   return_full_text=False, max_length=512, truncation=True)
gen_sqls = [x[0]['generated_text'] for x in gen_sqls] #위에 코드 블럭에서 보이듯이 generated_text 라는 id에 답변들이 저장되어 있음
df['gen_sql'] = gen_sqls #답변들 저장해줌

# 평가를 위한 requests.jsonl 생성
eval_filepath = "text2sql_evaluation.jsonl"
make_requests_for_gpt_evaluation(df, eval_filepath) #gpt에 평가 받을 데이터를 만들어 줌

In [ ]:
# GPT-4 평가 수행
!python api_request_parallel_processor.py \
--requests_filepath requests/{eval_filepath}  \
--save_filepath results/{eval_filepath} \
--request_url https://api.openai.com/v1/chat/completions \
--max_requests_per_minute 200 \
--max_tokens_per_minute 100000 \
--token_encoding_name cl100k_base \
--max_attempts 5 \
--logging_level 20

In [ ]:
base_eval = change_jsonl_to_csv(f"results/{eval_filepath}", "results/yi_ko_6b_eval.csv", "prompt", "resolve_yn")
base_eval['resolve_yn'] = base_eval['resolve_yn'].apply(lambda x: json.loads(x)['resolve_yn'])
num_correct_answers = base_eval.query("resolve_yn == 'yes'").shape[0]
num_correct_answers

## 예제 6.9. 학습 데이터 불러오기

In [ ]:
from datasets import load_dataset

df_sql = load_dataset("shangrilar/ko_text2sql", "origin")["train"] #미세 조정을 할 생각이므로 train 데이터를 가져와서 위와 똑같이 해줌
df_sql = df_sql.to_pandas()
df_sql = df_sql.dropna().sample(frac=1, random_state=42) #데이터를 정리하지 않아서 결측치가 있을 수 있으므로 드랍해줌 sample() 함수는 데이터프레임의 행을 무작위로 섞어 샘플링하는 함수입니다.
df_sql = df_sql.query("db_id != 1") #db_id=1은 위에서 test로 사용하기로 했으니까 다른 제외해버림

for idx, row in df_sql.iterrows():
  df_sql.loc[idx, 'text'] = make_prompt(row['context'], row['question'], row['answer']) #평가 할때 와 다르게 answer이 들어감
#data 폴더 만들어서 저장
!mkdir data
df_sql.to_csv('data/train.csv', index=False)

## 예제 6.10. 미세 조정 명령어

In [ ]:
base_model = 'beomi/Yi-Ko-6B'
finetuned_model = 'yi-ko-6b-text2sql'
#여기서 메모리 에러가 나면 배치 사이즈를 줄인다.
#또 다른 문제는 A100 GPU(코랩 유료 결제 버전 GPU)를 사용하면 1시간 걸리는데
#무료 GPU를 사용하면 8~9시간이 걸린다고한다.
!autotrain llm \
--train \
--model {base_model} \
--project-name {finetuned_model} \
--data-path data/ \
--text-column text \
--lr 2e-4 \
--batch-size 8 \
--epochs 1 \
--block-size 1024 \
--warmup-ratio 0.1 \
--lora-r 16 \
--lora-alpha 32 \
--lora-dropout 0.05 \
--weight-decay 0.01 \
--gradient-accumulation 8 \
--mixed-precision fp16 \
--use-peft \
--quantization int4 \
--trainer sft

## 예제 6.11. LoRA 어댑터 결합 및 허깅페이스 허브 업로드

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, PeftModel

model_name = start_model
device_map = {"": 0}

# LoRA와 기초 모델 파라미터 합치기
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 허깅페이스 허브에 모델 및 토크나이저 저장
model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

## 예제 6.12. 미세 조정한 모델로 예시 데이터에 대한 SQL 생성

In [ ]:
model_id = "shangrilar/yi-ko-6b-text2sql"
hf_pipe = make_inference_pipeline(model_id)#학습한 모델로 변경해서 파이프라인 생성

hf_pipe(example, do_sample=False,
       return_full_text=False, max_length=1024, truncation=True)
# SELECT COUNT(*) FROM players WHERE username LIKE '%admin%';

## 예제 6.13. 미세 조정한 모델 성능 측정

In [ ]:
# sql 생성 수행
gen_sqls = hf_pipe(df['prompt'].tolist(), do_sample=False,
                   return_full_text=False, max_length=1024, truncation=True)
gen_sqls = [x[0]['generated_text'] for x in gen_sqls]
df['gen_sql'] = gen_sqls

# 평가를 위한 requests.jsonl 생성
ft_eval_filepath = "text2sql_evaluation_finetuned.jsonl"
make_requests_for_gpt_evaluation(df, ft_eval_filepath)

# GPT-4 평가 수행
!python api_request_parallel_processor.py \
  --requests_filepath requests/{ft_eval_filepath} \
  --save_filepath results/{ft_eval_filepath} \
  --request_url https://api.openai.com/v1/chat/completions \
  --max_requests_per_minute 300 \
  --max_tokens_per_minute 100000 \
  --token_encoding_name cl100k_base \
  --max_attempts 5 \
  --logging_level 20

In [ ]:
ft_eval = change_jsonl_to_csv(f"results/{ft_eval_filepath}", "results/yi_ko_6b_eval.csv", "prompt", "resolve_yn")
ft_eval['resolve_yn'] = ft_eval['resolve_yn'].apply(lambda x: json.loads(x)['resolve_yn'])
num_correct_answers = ft_eval.query("resolve_yn == 'yes'").shape[0]
num_correct_answers